In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import os
import tqdm
def word2vec(words):
    '''
    Convert the words to word vectors
    '''
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = []
    for word in tqdm.tqdm(words):
        embedding = model.encode([word])
        embeddings.append(embedding)
    return np.concatenate(embeddings, axis=0)

/home/lixing/miniconda3/envs/har/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import string
def text_process(text):
    text = text.replace("#C C ", "").replace("#C", "").replace("#O", "").replace("#unsure", "").strip().strip(string.punctuation).lower()[:128]
    text = text.replace("summary", "")
    return text
metadata_dir = './ego4d/soundingaction/'
meta_file = 'egoclip_val_clean_audio_halo.csv'
metadata = pd.read_csv(metadata_dir + meta_file, header=0, sep='\t')
clip_text = metadata['clip_text'].tolist()
clip_text = [text_process(text) for text in clip_text]
clip_embeddings = word2vec(clip_text)

100%|██████████| 29989/29989 [00:57<00:00, 523.81it/s]


In [3]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
def find_clusters(vectors, n_clusters=50, top_n=50):
    '''
    Find the top n_clusters clusters in the embeddings.
    Each cluster returns the top 50 words closest to the cluster center.
    
    Args:
        embeddings: List or array of tuples (word, embedding), where embedding is a numpy array.
        n_clusters: Number of clusters to form (default: 50).
        top_n: Number of words to select per cluster (default: 50).
    
    Returns:
        clusters: List of lists, where each inner list contains the top_n words for a cluster.
    '''
    # Separate words and their embeddings
    vectors = np.array(vectors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(vectors)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    # Initialize clusters
    clusters = [[] for _ in range(n_clusters)]
    
    # Assign words to clusters
    for i, (vector, label) in enumerate(zip(vectors, labels)):
        clusters[label].append((i, vector))
    
    # For each cluster, select top_n words closest to the centroid
    top_indices_per_cluster = []
    for cluster_idx, cluster in enumerate(clusters):
        if not cluster:  # Skip empty clusters
            continue
        cluster_indices, cluster_vectors = zip(*cluster)
        cluster_vectors = np.array(cluster_vectors)
        
        # Compute Euclidean distances to the centroid
        distances = cdist(cluster_vectors, [centroids[cluster_idx]], metric='euclidean').flatten()
        
        # Get indices of top_n closest words
        top_indices = np.argsort(distances)[:min(top_n, len(cluster))]
        top_indices = [cluster_indices[idx] for idx in top_indices]
        top_indices_per_cluster.append(top_indices)
    
    return top_indices_per_cluster

In [16]:
# find the top50 clusters for clip_embeddings, each cluster we use the top 50 words
positive = metadata['positive'].tolist()
# the number of positive words (1) and negative words (0)
positive_count = sum(positive)
negative_count = len(positive) - positive_count
print(f'Number of positive words: {positive_count}, Number of negative words: {negative_count}')

# split the embeddings into positive and negative
positive_embeddings = clip_embeddings[np.array(positive) == 1]; positive_idx = np.where(np.array(positive) == 1)[0]
negative_embeddings = clip_embeddings[np.array(positive) == 0]; negative_idx = np.where(np.array(positive) == 0)[0]

top_words_per_cluster = find_clusters(positive_embeddings, n_clusters=50, top_n=200)
csv_name = './ego4d/soundingaction/cluster50_positive.csv'
csv_dataframe = []; activity_names = []
for i, cluster in enumerate(top_words_per_cluster):
    cluster_idx = positive_idx[cluster]
    cluster_center = cluster_idx[0]; cluster_center_text = clip_text[cluster_center]
    activity_names.append(cluster_center_text)
    cluster_meta = metadata.iloc[cluster_idx]
    # add a new column to the dataframe to indicate the activity index
    cluster_meta = cluster_meta.copy()
    cluster_meta['activity_index'] = i
    cluster_meta['activity_name'] = cluster_center_text
   
    csv_dataframe.append(cluster_meta)
csv_dataframe = pd.concat(csv_dataframe, ignore_index=True)
csv_dataframe.to_csv(csv_name, index=False, sep='\t')
print(f'Saved top 50 clusters of positive words to {csv_name}')
np.savetxt('./ego4d/soundingaction/cluster50_activity_names.txt', activity_names, fmt='%s')

# top_words_per_cluster = find_clusters(negative_embeddings, n_clusters=50, top_n=-1)
# csv_name = './ego4d/soundingaction/cluster50_50_negative.csv'
# csv_dataframe = []
# for i, cluster in enumerate(top_words_per_cluster):             
#     cluster_idx = negative_idx[cluster]
#     cluster_meta = metadata.iloc[cluster_idx]
#     cluster_meta = cluster_meta.copy()
#     cluster_meta['activity_index'] = i
#     csv_dataframe.append(cluster_meta)
# csv_dataframe = pd.concat(csv_dataframe, ignore_index=True)
# csv_dataframe.to_csv(csv_name, index=False, sep='\t')
# print(f'Saved top 50 clusters of negative words to {csv_name}') 

Number of positive words: 16380.0, Number of negative words: 13609.0
Saved top 50 clusters of positive words to ./ego4d/soundingaction/cluster50_positive.csv


In [3]:
from ego4d.ego4d_dataset import Ego4D_Action2Sound
import soundfile as sf
import numpy as np
import os
import tqdm

dataset = Ego4D_Action2Sound(folder='../dataset/ego4d/v2/', window_sec=10, modal=['audio', 'imu', 'narration'],
                                     csv='ego4d/soundingaction/cluster50_positive.csv')
output_dir = '../dataset/ego4d/positive/'
os.makedirs(output_dir, exist_ok=True)
for i in tqdm.tqdm(range(len(dataset))):
    sample = dataset[i]
    audio, imu, activity = sample['audio'], sample['imu'], sample['text']
    scenario_name = sample['scenario_name']
   
    activity_folder = os.path.join(output_dir, activity)
    fname = f"{scenario_name}_{i}"

    os.makedirs(activity_folder, exist_ok=True)
    audio_path = os.path.join(activity_folder, fname + '.wav')
    sf.write(audio_path, audio, 16000)
    imu_path = os.path.join(activity_folder, fname + '.npy')
    np.save(imu_path, imu)

Loading sound annotation from ego4d/soundingaction/cluster50_positive.csv
Preparing training data


100%|██████████| 141/141 [00:00<00:00, 67120.29it/s]


Number of Scenario 41
There are 2374 windows to process.


  0%|          | 4/2374 [00:00<01:02, 37.72it/s]

holds the paper
moves the hand
puts a card on the table
puts a card on the table
puts a card on the table
puts a card on the table
puts a card on the table


...

puts clothes on the hanger stand
puts clothes on the hanger stand
puts clothes on the hanger stand
puts clothes on the hanger stand
puts clothes on the hanger stand
puts clothes on the hanger stand
puts clothes on the hanger stand
puts clothes on the hanger stand
tightens the nut


 99%|█████████▉| 2360/2374 [00:46<00:00, 49.45it/s]

opens the door
puts clothes on the hanger stand
puts off the rice grinder with her right hand
cuts the grass
cuts the grass
cuts the grass
cuts the grass
moves forward
moves forward
moves forward
moves forward
opens the door


100%|██████████| 2374/2374 [00:46<00:00, 50.54it/s]

tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
tightens the nut
